在载入、合并、准备数据集之后，可能需要计算分组统计或者数据透视表用于报告或可视化的目的

在本章，将学习：  
* 使用一个或多个键（以函数、数组或DataFrame列名的形式）将pandas对象拆分为多块  
* 计算组汇总统计信息，如计数、平均值或标准偏差或用户定义的函数   
* 应用组内变换或其他操作，如标准化、线性回归、排位或子集选择  
* 计算数据透视表和交叉表  
* 执行分位数分析和其他统计组分析  

## GroupBy机制

In [ ]:
#根据key1标签计算data1列的均值
grouped = df['data1'].groupby(df['key1'])   #其中一种方法，访问data1并使用key1列调用groupby方法
                                            #这个例子中分组键是Series,尽管分组键也可以是正确长度的任何数组

In [ ]:
df.groupby('key1').mean()  #默认情况下，所有数值列都可以聚合

In [ ]:
df.groupby(['key1', 'key2']).size()  #size方法返回一个包含组大小信息的Series

__遍历各分组__

GroupBy对象支持迭代，会生成一个包含组名和数据块的2维元组序列

In [ ]:
for name, group in df.groupby('key1'):
    print(name)
    print(value)
    
for dtype, group in df.groupby(df.dtype, axis=1):   #根据dtype对列分组
    print(dtype)
    print(group)

__选择一列或所有列的子集__

In [ ]:
df.groupby('key1')['data1'].mean()   #返回分组的DataFrame
df.groupby('key1')[['data2']].mean()   #返回分组的Series

__使用字典和Seires分组__

In [ ]:
#mapping是个字典，有各个列的对应名称
by_column = people.groupby(mapping, axis=1)  #有了各列的分组对应关系，把各列按组相加
by_column.sum()

__使用函数分组__

与使用字典或Series分组相比，使用Python函数是定义分组关系的一种更为通用的方式。  
作为分组键传递的函数会按照每个索引值调用一次，同时返回值会被用作分组名称。

In [ ]:
people.groupby(len).sum()

__根据索引层级分组__

In [ ]:
hier_df.groupby(level='city', axis=1).count()

## 数据聚合

In [ ]:
df.groupby('key1')['data1'].quantile(0.9)

In [ ]:
grouped.agg(peak_to_peak)  #要使用自己的聚合函数，需要将函数传递给aggregate或agg方法

__逐列及多函数应用__

根据各列同时使用多个函数进行聚合

In [ ]:
grouped = tips.groupby(['day', 'smoker'])
grouped_pct = grouped['tip_pct']
grouped_pct.agg('mean')   #可以将函数名以字符串形式传递

In [ ]:
grouped_pct.agg(['mean', 'std', peak_to_peak])   #如果传递的是函数或者函数名的列表，会获得一个列名是这些函数名的DataFrame

In [ ]:
grouped_pct.agg([('foo', 'mean'), ('bar', np.std)])   #如果传递的是(name, function)元组的列表，每个元组的第一个元素将作为DataFrame的列名

心得：如果grouped_pct选的是grouped的几个列，就返回这几个列和函数的笛卡尔积DataFrame

那如果想在不同的列用不同的函数怎么办？ 　　答：用字典

In [ ]:
grouped.agg({'tip_pct' : ['min', 'max', 'mean', 'std'],
             'size': 'sum'})

__返回不含行索引的聚合数据__

In [ ]:
tips.groupby(['day', 'smoker'], as_index=False).mean()    #禁止分组键作为索引

###  应用：通用拆分-应用-联合

GroupBy方法最常见的目的是apply（应用）

In [ ]:
#先写一个可以在特定列中选中最大值所在行的函数 top(df, n=5, column='tip_pct')
tips.groupby(['smoker', 'day']).apply(top)

__压缩分组键__

In [ ]:
tips.groupby('smoker', group_keys=False).apply(top)    #之前例子所得到的对象具有分组键所形成的分层索引以及每个原始对象的索引。
                                                        #通过想groupby传递group_keys=False来禁用这个功能

__分位数与桶分析__

cut返回的Categorical对象可以直接传递给groupby。所以可以计算出data2列的一个统计值集合

In [ ]:
quartiles = pd.cut(frame.data1, 4)

#定义一个函数get_stats(group)

grouped = frame.data2.groupby(quartiles)

__示例：使用指定分组值填充缺失值__

可以给每个组设置不同的填充缺失值的方法

__示例：随机采样与排列__

__示例：分组加权平均和相关性__

在groupby的拆分-应用、联合的范式下，DataFrame的列间操作或两个Series之间的操作，例如分组加权平均是可以做到的；还有股票间年度相关性

__示例：逐组先行回归__